In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
import pandas as pd
import os
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# **Veri artırma için ImageDataGenerator**

In [ ]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# **GoogleNet Mimarisi**

In [ ]:
def inception_module(x, filters):
    f1, f2_in, f2_out, f3_in, f3_out, f4_out = filters

    conv1 = layers.Conv2D(f1, (1, 1), padding='same', activation='relu')(x)

    conv3 = layers.Conv2D(f2_in, (1, 1), padding='same', activation='relu')(x)
    conv3 = layers.Conv2D(f2_out, (3, 3), padding='same', activation='relu')(conv3)

    conv5 = layers.Conv2D(f3_in, (1, 1), padding='same', activation='relu')(x)
    conv5 = layers.Conv2D(f3_out, (5, 5), padding='same', activation='relu')(conv5)

    pool = layers.MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
    pool = layers.Conv2D(f4_out, (1, 1), padding='same', activation='relu')(pool)

    return layers.concatenate([conv1, conv3, conv5, pool], axis=-1)

def googlenet_model(input_shape, num_classes):
    input_layer = layers.Input(shape=input_shape)

    x = layers.Conv2D(64, (7, 7), strides=(2, 2), padding='same', activation='relu')(input_layer)
    x = layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    x = layers.Conv2D(64, (1, 1), padding='same', activation='relu')(x)
    x = layers.Conv2D(192, (3, 3), padding='same', activation='relu')(x)
    x = layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

    x = inception_module(x, [64, 96, 128, 16, 32, 32])
    x = inception_module(x, [128, 128, 192, 32, 96, 64])
    x = layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

    x = inception_module(x, [192, 96, 208, 16, 48, 64])
    x = inception_module(x, [160, 112, 224, 24, 64, 64])
    x = inception_module(x, [128, 128, 256, 24, 64, 64])
    x = inception_module(x, [112, 144, 288, 32, 64, 64])
    x = inception_module(x, [256, 160, 320, 32, 128, 128])
    x = layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

    x = inception_module(x, [256, 160, 320, 32, 128, 128])
    x = inception_module(x, [384, 192, 384, 48, 128, 128])

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.4)(x)
    output_layer = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(input_layer, output_layer)
    return model

# **Veri işlemesi ve train-test ayrımı**

In [ ]:
data_dir = '/kaggle/input/crop-diseases-classification/Data/train_images'
labels_csv_path = '/kaggle/input/crop-diseases-classification/Data/train.csv'

labels_df = pd.read_csv(labels_csv_path)

image_filenames = labels_df['image_id'].tolist()
labels = labels_df['label'].tolist()

image_paths = [os.path.join(data_dir, filename) for filename in image_filenames]

valid_image_paths = []
valid_labels = []

for index, row in labels_df.iterrows():
    image_path = os.path.join(data_dir, row['image_id'])
    if os.path.exists(image_path):
        valid_image_paths.append(image_path)
        valid_labels.append(row['label'])

if len(valid_image_paths) == 0:
    raise ValueError("No valid image paths found. Please check your file paths and ensure the images exist.")

train_paths, test_paths, train_labels, test_labels = train_test_split(valid_image_paths, valid_labels, test_size=0.2, random_state=42)

def preprocess_image(image_path, label, img_size=(224, 224)):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, img_size)
    image = image / 255.0
    return image, label


# **Görüntü verilerini artırmak**

In [ ]:
def augment(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=0.1)
    image = tf.image.random_contrast(image, lower=0.9, upper=1.1)
    return image, label

# **Veri seti oluşturma ve ön işleme**

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_paths, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_paths, test_labels))

train_dataset = train_dataset.map(lambda x, y: preprocess_image(x, y), num_parallel_calls=tf.data.AUTOTUNE)
train_dataset = train_dataset.map(lambda x, y: augment(x, y), num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = test_dataset.map(lambda x, y: preprocess_image(x, y), num_parallel_calls=tf.data.AUTOTUNE)

BATCH_SIZE = 16
SHUFFLE_BUFFER_SIZE = 1000

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# **Model Oluşturma ve Derleme**

In [ ]:
input_shape = (224, 224, 3)
num_classes = len(set(valid_labels))

model = googlenet_model(input_shape, num_classes)
initial_learning_rate = 0.0001

optimizer = tf.keras.optimizers.Adam(learning_rate=initial_learning_rate)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

for image_batch, label_batch in train_dataset.take(1):
    print(f"Image batch shape: {image_batch.shape}")
    print(f"Label batch shape: {label_batch.shape}")

for image_batch, label_batch in test_dataset.take(1):
    print(f"Image batch shape: {image_batch.shape}")
    print(f"Label batch shape: {label_batch.shape}")


# **Geri çağırma (callback)**

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)

# **Modeli eğitme**

In [ ]:
history = model.fit(train_dataset, epochs=30, validation_data=test_dataset, callbacks=[early_stopping, reduce_lr])

# **Modeli test veri kümesi üzerinde değerlendirme**

In [ ]:
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

def plot_history(history):
    plt.figure(figsize=(12, 4))

    # Loss grafiği
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    # Accuracy grafiği
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.show()

# Metrikleri çiz
plot_history(history)

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
y_pred = model.predict(test_dataset)
y_pred_classes = np.argmax(y_pred, axis=1)

# Gerçek etiketleri al
y_true = np.concatenate([y for x, y in test_dataset], axis=0)

# Doğruluk (Accuracy)
accuracy = accuracy_score(y_true, y_pred_classes)
print(f"Accuracy: {accuracy:.4f}")

# Kesinlik (Precision)
precision = precision_score(y_true, y_pred_classes, average='weighted')
print(f"Precision: {precision:.4f}")

# Geri Çağırma (Recall)
recall = recall_score(y_true, y_pred_classes, average='weighted')
print(f"Recall: {recall:.4f}")

# F-ölçüsü (F1-Score)
f1 = f1_score(y_true, y_pred_classes, average='weighted')
print(f"F1-Score: {f1:.4f}")
